## Setup

In [36]:
from py2neo import Graph, Node, Relationship
import glob, os, time
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

## Connect to graph

In [37]:
public_address = '54.174.175.98'
graph = Graph('bolt://{}:7687'.format(public_address), auth=('neo4j','myneo'))

n_nodes = graph.database.primitive_counts['NumberOfNodeIdsInUse']
n_relationships = graph.database.primitive_counts['NumberOfRelationshipIdsInUse']
print("Connected to graph database with {:,} nodes and {:,} relationships!".format
     (n_nodes, n_relationships))

DEBUG:neo4j.bolt:~~ [RESOLVE] Address(host='54.174.175.98', port=7687)
DEBUG:neo4j.bolt:~~ [RESOLVED] Address(host='54.174.175.98', port=7687) -> ('54.174.175.98', 7687)
DEBUG:neo4j.bolt:~~ [CONNECT] ('54.174.175.98', 7687)
DEBUG:neo4j.bolt:C: [HANDSHAKE] 0x6060B017 [2, 1, 0, 0]
DEBUG:neo4j.bolt:S: [HANDSHAKE] 2
DEBUG:neo4j.bolt:C: INIT ('py2neo/4.1.0 neo4j-python/1.6.3 Python/3.6.7-final-0 (linux)', {...})
DEBUG:neo4j.bolt:S: SUCCESS ({'server': 'Neo4j/3.5.7'})
DEBUG:neo4j.bolt:~~ [CLOSE]
DEBUG:neo4j.bolt:C: RUN ("CALL dbms.queryJmx('')", {})
DEBUG:neo4j.bolt:C: PULL_ALL ()
DEBUG:neo4j.bolt:S: SUCCESS ({'result_available_after': 27, 'fields': ['name', 'description', 'attributes']})
DEBUG:neo4j.bolt:S: RECORD * 2
DEBUG:neo4j.bolt:S: RECORD * 2
DEBUG:neo4j.bolt:S: RECORD * 7
DEBUG:neo4j.bolt:S: RECORD * 8
DEBUG:neo4j.bolt:S: RECORD * 2
DEBUG:neo4j.bolt:S: RECORD * 1
DEBUG:neo4j.bolt:S: RECORD * 6
DEBUG:neo4j.bolt:S: SUCCESS ({'result_consumed_after': 1, 'type': 'r'})
DEBUG:neo4j.bolt:C:

Connected to graph database with 370,041,408 nodes and 14,502,760 relationships!


## Setup workspace

In [38]:
## In Neo4j 3.5+, the directory is automatically set to /import
data_dir = '' 
scipy_data_dir = '/tmp/data/'

print("Neo4j data directory set to `{}`.".format(data_dir))
print("SciPy data directory set to `{}`.".format(scipy_data_dir))

# MAG files
mag_venues_file = data_dir + 'oagtwo/mag_venues.txt'

n_mag_papers_files = len(glob.glob(scipy_data_dir + 'oagtwo/mag_papers_*.txt'))
mag_papers_files = [data_dir + 'oagtwo/mag_papers_{}.txt'.format(i) for i in range(n_mag_papers_files)]

n_mag_authors_files = len(glob.glob(scipy_data_dir + 'oagtwo/mag_authors_*.txt'))
mag_authors_files = [data_dir + 'oagtwo/mag_authors_{}.txt'.format(i) for i in range(n_mag_authors_files)]

n_magv1_papers_files = len(glob.glob(scipy_data_dir + 'oagone/mag_papers_*.txt'))
mag_v1_papers_files = [data_dir + 'oagone/mag_papers_{}.txt'.format(i) for i in range(n_magv1_papers_files)]

print("\nMAG files:")
print("\tVenue files: {}".format(1))
print("\tPapers files: {}".format(n_mag_papers_files))
print("\tAuthors files: {}".format(n_mag_authors_files))
print("\tMAGv1 papers files: {}".format(n_magv1_papers_files))

# AMiner files
aminer_venues_file = data_dir + 'oagtwo/aminer_venues.txt'

n_aminer_papers_files = len(glob.glob(scipy_data_dir + 'oagtwo/aminer_papers_*.txt'))
aminer_papers_files = [data_dir + 'oagtwo/aminer_papers_{}.txt'.format(i) for i in range(n_aminer_papers_files)]

n_aminer_authors_files = len(glob.glob(scipy_data_dir + 'oagtwo/aminer_authors_*.txt'))
aminer_authors_files = [data_dir + 'oagtwo/aminer_authors_{}.txt'.format(i) for i in range(n_aminer_authors_files)]

n_aminer_v1_papers_files = len(glob.glob(scipy_data_dir + 'oagone/aminer_papers_*.txt'))
aminer_v1_papers_files = [data_dir + 'oagone/aminer_papers_{}.txt'.format(i) for i in range(n_aminer_v1_papers_files)]
      
print("\nAMiner files:")
print("\tVenue files: {}".format(1))
print("\tPapers files: {}".format(n_aminer_papers_files))
print("\tAuthors files: {}".format(n_aminer_authors_files))
print("\tAMinerv1 papers files: {}".format(n_aminer_v1_papers_files))

# Linking pairs
venue_linking_pairs = 'oagtwo/venue_linking_pairs.txt'
paper_linking_pairs = 'oagtwo/paper_linking_pairs.txt'
author_linking_pairs = 'oagtwo/author_linking_pairs.txt'

def run_query(query, graph, print_query=False, run_query=True, print_only=False):
    if print_only: 
        print_query = True
        run_query = False
    start_time = time.time()
    if print_query:
        print(query)
    if run_query:
        graph.run(query)
    end_time = time.time()
    seconds_elapsed = end_time-start_time
    minutes_elapsed = (end_time-start_time)/60
    print("Query completed in {:.2f} seconds.".format(seconds_elapsed))

top_5 = ['Cell', 'Nature', 'Nature Biotechnology','Proceedings of the National Academy of Sciences of the United States of America','Science']
top_10 = ['Cell', 'Nature', 'Nature Biotechnology','Proceedings of the National Academy of Sciences of the United States of America','Science', 'Journal of the American Chemical Society', 'JAMA', 'The New England Journal of Medicine', 'Nature Genetics', 'Neuron']
top_42 = ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']
top_42_mag_ids = ['172573765','168778273','3880285','103870658','49861241','200071133','106963461','139253143','151741590','175814247','67393510','2486275289','41063453','24207032','177533077','96410829','110447773','2298632','203256638','202193212','154343897','45757444','162030435','62468778','128124174','2004986','199671312','85134406','111155417','125754415','127916151','140251998','207313999','137905309','168522863','114430552','25671836','51309854','155937366','22947807','127827428','137773608']
top_42_aminer_ids = ['5451a5c4e0cf0b02b5f3821a','5451a5c5e0cf0b02b5f396be','5451a5c6e0cf0b02b5f3984f','5451a5c4e0cf0b02b5f38892','5451a5c4e0cf0b02b5f38092','5451a5c5e0cf0b02b5f39630','5451a5c7e0cf0b02b5f3ac13','5451a57ae0cf0b02b5f2dd9f','5451a5c4e0cf0b02b5f380b3','5451a5c4e0cf0b02b5f3935a','5451a5c4e0cf0b02b5f3844b','5451a5c6e0cf0b02b5f3986c','5451a5c5e0cf0b02b5f396fe','5451a5c4e0cf0b02b5f381da','5451a5c4e0cf0b02b5f38910','5451a5c4e0cf0b02b5f38093','5451a5c9e0cf0b02b5f3bcf1','5451a5c4e0cf0b02b5f388fa','5451a5c6e0cf0b02b5f3977e','5451a5c4e0cf0b02b5f38030','5451a5c4e0cf0b02b5f39329','5451a5c4e0cf0b02b5f38771','5451a5c4e0cf0b02b5f37fc6','5451a5c5e0cf0b02b5f3967a','5451a5c4e0cf0b02b5f384ad','5451a5c4e0cf0b02b5f39454','5451a5c4e0cf0b02b5f38097','5451a5c4e0cf0b02b5f3827c','5451a5c6e0cf0b02b5f39776','5451a5c4e0cf0b02b5f37c20','5451a57ae0cf0b02b5f2de43','5451a5c6e0cf0b02b5f3978e','5451a5c4e0cf0b02b5f3886a','5451a5c4e0cf0b02b5f37fc0','5451a5c5e0cf0b02b5f3970e','5451a5c5e0cf0b02b5f396c9','5451a5c5e0cf0b02b5f3973f','5451a5c4e0cf0b02b5f384c5','5451a5c6e0cf0b02b5f39887','5451a5c4e0cf0b02b5f387a9','5451a5c4e0cf0b02b5f38091','5451a5c4e0cf0b02b5f37dd5']
top_42_ids = top_42_mag_ids + top_42_aminer_ids

print(top_42_ids)

Neo4j data directory set to ``.
SciPy data directory set to `/tmp/data/`.

MAG files:
	Venue files: 1
	Papers files: 11
	Authors files: 13
	MAGv1 papers files: 167

AMiner files:
	Venue files: 1
	Papers files: 15
	Authors files: 20
	AMinerv1 papers files: 155
['172573765', '168778273', '3880285', '103870658', '49861241', '200071133', '106963461', '139253143', '151741590', '175814247', '67393510', '2486275289', '41063453', '24207032', '177533077', '96410829', '110447773', '2298632', '203256638', '202193212', '154343897', '45757444', '162030435', '62468778', '128124174', '2004986', '199671312', '85134406', '111155417', '125754415', '127916151', '140251998', '207313999', '137905309', '168522863', '114430552', '25671836', '51309854', '155937366', '22947807', '127827428', '137773608', '5451a5c4e0cf0b02b5f3821a', '5451a5c5e0cf0b02b5f396be', '5451a5c6e0cf0b02b5f3984f', '5451a5c4e0cf0b02b5f38892', '5451a5c4e0cf0b02b5f38092', '5451a5c5e0cf0b02b5f39630', '5451a5c7e0cf0b02b5f3ac13', '5451a57ae0cf

# Import functions

In [1]:
def add_constraints(print_only=False):
    logging.info("Adding uniqueness constraints")
    contraint_pairs = [('n:Quanta', 'n.id'),  
                       ('a:Author', 'a.id'),
                       ('v:Venue', 'v.id'),
                       ('t:Tag', 't.cleanName')]
    
    for n, p in contraint_pairs:
        query = "CREATE CONSTRAINT ON ({}) ASSERT {} IS UNIQUE;".format(n,p)
        run_query(query, graph, print_only=print_only)
        
def add_indices(print_only=False):
    logging.info("Adding indexes")
    indices = [':Quanta(cleanName)',
               ':Quanta(idv1)',
               ':Venue(cleanName)', 
               ':Venue(matchingId)',
               ':Year(year)',
               ':Author(cleanName)',
               ':Organization(cleanName)']
    
    for index in indices:
        query = "CREATE INDEX ON {};".format(index)
        run_query(query, graph, print_only=print_only)

## (:Year)
def add_years(min_year, max_year, print_only=False):
    logging.info("Adding (:Year)")
    query = """
    UNWIND range({}, {}) as yr
    MERGE (y:Year {{year: yr}})
    """.format(min_year, max_year)
    run_query(query, graph, print_only=print_only)
    
## (:Venue)
def add_venues(venues_file, source_label='', print_only=False):
    logging.info("Adding (:Venue)".format(source_label))
    query = """ 
    CALL apoc.load.json('{venue_file}') YIELD value AS q
    CREATE (v:Venue{source_label} {{id:q.id}})
        SET v.journalId = q.JournalId, 
            v.conferenceId = q.ConferenceId, 
            v.matchingId = coalesce(q.JournalId, q.ConferenceId, q.id),
            v.name = q.DisplayName, 
            v.normalizedName = q.NormalizedName,
            v.cleanName = apoc.text.clean(q.DisplayName);
    """.format(source_label=source_label, venue_file=venues_file)
    run_query(query, graph, print_only=print_only)

## (:Quanta)
def add_quanta(papers_files, source_label='', print_only=False):
    logging.info("Adding (:Quanta{})".format(source_label))
    for file_name in papers_files:
        print('Importing {}'.format(file_name))
        query = """
        CALL apoc.periodic.iterate(
        "CALL apoc.load.json('{paper_file}') YIELD value AS q RETURN q",
        "WHERE 
            (q IS NOT NULL) AND 
            (size(q.title) <= 3000) AND
            exists(q.venue.id) AND
            (q.venue.id IN {venue_ids})
         MERGE (p:Quanta{source_label} {{id:q.id}})
            SET p.title = q.title, 
                p.cleanName = apoc.text.clean(q.title),
                p.year = q.year, 
                p.citations = q.n_citation, 
                p.docType = q.doc_type, 
                p.language = q.lang, 
                p.publisher = q.publisher, 
                p.doi = q.doi, 
                p.pdf = q.pdf, 
                p.volume = q.volume, 
                p.issue = q.issue,
                p.abstract = q.abstract;",
        {{batchSize:10000, iterateList:true, parallel:true}});
        """.format(paper_file=file_name, source_label=source_label, venue_ids=top_42_ids)
        run_query(query, graph, print_only=print_only)
        
## (:Quanta {idv1})")
def add_v1_data(v1_papers_files, source_label='', print_only=False):
    logging.info("Adding (:Quanta{}) from OAGv1".format(source_label))
    for file_name in v1_papers_files:
        query = """
        CALL apoc.periodic.iterate(
            "CALL apoc.load.json('{papers_file}') YIELD value AS q RETURN q",
            "MATCH (p:Quanta{source_label})
             WHERE (p.cleanName = apoc.text.clean(q.title))
                 AND ((p.doi = q.doi) OR ((p.volume = q.volume) AND (p.issue = q.issue)))
             SET p.idv1 = q.id, p.abstract=q.abstract",
            {{batchSize:10000, iterateList:true, parallel:false}});
        """.format(source_label=source_label, papers_file=file_name)
        run_query(query, graph, print_only=print_only)
        
def merge_quanta_on_cleanname(source_label='', print_only=False):
    logging.info("Merging (q:Quanta{}) on q.cleanName".format(source_label))
    
    # First match in fine-grained way
    query = """
    CALL apoc.periodic.iterate(
        "MATCH (q:Quanta{source_label}) 
         WHERE EXISTS(q.cleanName)
         WITH 
             q.cleanName as cleanName, 
             q.year as year, 
             q.publisher as publisher, 
             q.doi as doi,
             q.volume as volume,
             q.issue as issue,
             COLLECT(q) as nodeList, 
             COUNT(q) as count
         WHERE count>=2
         RETURN nodeList",
        "CALL apoc.refactor.mergeNodes(nodeList, 
         {{properties:'discard', mergeRels:true}})
         YIELD node
         RETURN 'none'",
        {{batchSize:10000, iterateList:true, parallel:true}});
    """.format(source_label=source_label)
    run_query(query, graph, print_only=print_only)
    
    # Now match in course-grained way
    query = """
    CALL apoc.periodic.iterate(
        "MATCH (q:Quanta{source_label}) 
         WHERE EXISTS(q.cleanName)
         WITH 
             q.cleanName as cleanName, 
             COLLECT(q) as nodeList, 
             COUNT(q) as count
         WHERE count>=2
         RETURN nodeList",
        "CALL apoc.refactor.mergeNodes(nodeList, 
         {{properties:'discard', mergeRels:true}})
         YIELD node
         RETURN 'none'",
        {{batchSize:10000, iterateList:true, parallel:true}});
    """.format(source_label=source_label)
    run_query(query, graph, print_only=print_only)
    
## (:Quanta)-[:PUBLISHED_IN]->(:Year)
def add_quanta_year_edges(papers_files, source_label='', print_only=False):
    logging.info("Adding (q:Quanta{})-[:PUBLISHED_IN]->(:Year)".format(source_label))
    for file_name in papers_files:
        query = """
        CALL apoc.periodic.iterate(
        "CALL apoc.load.json('{file}') YIELD value AS q RETURN q",
        "WHERE 
            (q IS NOT NULL) AND 
            (size(q.title) <= 3000) AND
            exists(q.venue.id) AND
            (q.venue.id IN {venues})
         MATCH (p:Quanta{source} {{id:q.id}})
         MATCH (y:Year {{year: q.year}})
         CREATE (p)-[:PUBLISHED_IN]->(y)",
        {{batchSize:10000, iterateList:true, parallel:false}})
        """.format(file=file_name, venues=top_42_ids, source=source_label)
        run_query(query, graph, print_only=print_only)
        
## (:Quanta)-[:PUBLISHED_IN]->(:Venue)
def add_quanta_venue_edges(papers_files, source_label='', print_only=False):
    logging.info("Adding (q:Quanta{})-[:PUBLISHED_IN]->(:Venue)".format(source_label))
    for file_name in papers_files:

        # First run on all venues WITH id's 
        query = """
        CALL apoc.periodic.iterate(
        "CALL apoc.load.json('{file}') 
         YIELD value AS q 
         RETURN q",
        "WHERE 
            (q IS NOT NULL) AND 
            (size(q.title) <= 3000) AND
            exists(q.venue.id) AND
            (q.venue.id IN {venues})
         MATCH (p:Quanta{source} {{id:q.id}} )
         MATCH (v:Venue {{matchingId:q.venue.id}} )
         CREATE (p)-[:PUBLISHED_IN]->(v)",
        {{batchSize:10000, iterateList:true, parallel:false}})        
        """.format(file=file_name, venues=top_42_ids, source=source_label)
        run_query(query, graph, print_only=print_only)
        
        # Second, run on all venues WITHOUT id's
        # This means that the :Venue is not in the graph
        # So we want to MERGE it in
#         query = """
#         CALL apoc.periodic.iterate(
#         "CALL apoc.load.json('{}') 
#          YIELD value AS q 
#          RETURN q",
#         "WHERE not(exists(q.venue.id)) AND exists(q.venue.raw) 
#          MATCH (p:Quanta {{id:q.id}} )
#          MERGE (v:Venue {{cleanName:apoc.text.clean(q.venue.raw)}} )
#          CREATE (p)-[:PUBLISHED_IN]->(v)",
#         {{batchSize:5000, iterateList:true, parallel:false}})        
#         """.format(file_name)            
#         run_query(query, graph, print_only=print_only)

## (:Author)
def add_authors(authors_files, source_label='', print_only=False):
    logging.info("Adding (:Author{})".format(source_label))
    for file_name in authors_files: 
        print('Importing {}'.format(file_name))
        query = """
        CALL apoc.periodic.iterate(
        "CALL apoc.load.json('{file}') YIELD value AS q RETURN q",
        "MERGE (a:Author{source} {{id:q.id}})
        SET a.name = q.name, 
            a.normalizedName = q.normalized_name, 
            a.cleanName = apoc.text.clean(q.normalized_name),
            a.position = q.position, 
            a.lastAffiliation = q.org,
            a.citations = q.n_citation, 
            a.numPublications = q.n_pubs, 
            a.hIndex=q.h_index",
        {{batchSize:10000, iterateList:true, parallel:true}});
        """.format(file=file_name, source=source_label)
        run_query(query, graph, print_only=print_only)
        
## (:Author)-[:AUTHORED]->(:Quanta)
def add_author_quanta_edges(authors_files, source_label='', print_only=False):
    logging.info("Adding (:Author{s})-[:AUTHORED]->(:Quanta{s})".format(s=source_label))
    for file_name in authors_files: 
        print('Creating authored relationships for {}'.format(file_name))
        query = """
        CALL apoc.periodic.iterate(
        "CALL apoc.load.json('{file}') YIELD value AS q RETURN q",
        "MATCH (a:Author{source} {{id:q.id}})
        UNWIND q.pubs as pub
        MATCH (p:Quanta{source} {{id:pub.i}})
        MERGE (a)-[r:AUTHORED]->(p)
        ON CREATE SET r.order = CASE WHEN pub.r IS NULL THEN NULL ELSE pub.r END",
        {{batchSize:10000, iterateList:true, parallel:false}});
        """.format(file=file_name, source=source_label)
        run_query(query, graph, print_only=print_only)
        
## (:Organization)
def add_organizations(authors_files, source_label='', print_only=False):
    logging.info("Adding (:Organization{s})".format(s=source_label))
    for file_name in authors_files: 
        query = """
        call apoc.periodic.commit("
        CALL apoc.load.json('{file}') 
        YIELD value AS q
        WITH q 
        LIMIT {{limit}}
        WHERE EXISTS(q.org)
        MATCH (:Author{s} {{id:q.id}})
        MERGE (o:Organization{s} {{cleanName: apoc.text.clean(q.org)}})
        ON CREATE SET o.name = q.org
        ", {{limit:50000}});
        """.format(file=file_name, s=source_label)
        run_query(query, graph, print_only=print_only)
        
## (:Author)-[:AFFILIATED]->(:Organization)
def add_author_organization_edges(authors_files, source_label='', print_only=False):
    logging.info("Adding (:Author{s})-[:AFFILIATED]->(:Organization{s})".format(s=source_label))
    for file_name in authors_files: 
        query = """
        CALL apoc.periodic.iterate(
            "CALL apoc.load.json('{file}') 
             YIELD value AS q 
             RETURN q",
            "WHERE q.org IS NOT NULL
             MATCH (a:Author{s} {{id: q.id}})
             MATCH (o:Organization{s} {{cleanName: apoc.text.clean(q.org)}})
             MERGE (a)-[:AFFILIATED]->(o);",
            {{batchSize:5000, iterateList:true, parallel:false}});
        """.format(file=file_name, s=source_label)
        run_query(query, graph, print_only=print_only)
        
## (:Quanta)-[:CITES]->(:Quanta)
def add_quanta_quanta_edges(v1_papers_files, source_label='', print_only=False):
    logging.info("Adding (:Quanta{s})-[:CITES]->(:Quanta{s})".format(s=source_label))
    for file_name in v1_papers_files:
        query = """
        CALL apoc.periodic.iterate("
            CALL apoc.load.json('{f}') 
            YIELD value AS q
            RETURN q
            ","
            MATCH (p:Quanta{s} {{idv1: q.id}})
            UNWIND q.references as ref
            WITH p, ref
            MATCH (b:Quanta{s} {{idv1: ref}})
            MERGE (p)-[:CITES]->(b)
            ",{{batchSize:10000, iterateList:true, parallel:false}});
        """.format(f=file_name, s=source_label)
        run_query(query, graph, print_only=print_only)
        
## (:Author)-[:COAUTHOR]-(:Author)
def add_author_author_edges(source_label='', print_only=False):
    logging.info("Adding (:Author)-[:COAUTHOR]-(:Author)".format(s=source_label))
    query = """
    CALL apoc.periodic.iterate(
        "MATCH (q:Quanta{s}) WHERE size((q)<-[:AUTHORED]-()) > 1 RETURN q",
        "WITH [(q)<-[:AUTHORED]-(a) | a] as coAuthors
        UNWIND coAuthors as first
        UNWIND coAuthors as second
        WITH first, second
        WHERE id(first) < id(second)
        MERGE (first)-[r:COAUTHOR]-(second)
        SET r.strength = CASE WHEN r.strength IS NULL THEN 1 ELSE r.strength + 1 END",
    {{batchSize:5000, iterateList:true, parallel:false}});
    """.format(s=source_label)
    run_query(query, graph, print_only=print_only)
    
    
## (:Tag)
def add_tags(authors_files, source_label='', print_only=False):
    logging.info("Adding (:Tag{s})".format(s=source_label))
    for file_name in authors_files: 
        query = """
        CALL apoc.periodic.iterate(
            "CALL apoc.load.json('{f}') 
             YIELD value AS q 
             RETURN q",
            "WITH q 
             WHERE exists(q.tags)
             MATCH (a:Author{s} {{id: q.id}})
             UNWIND q.tags as tag
             MERGE (t:Tag{s} {{cleanName: apoc.text.clean(tag.t)}})
             ON CREATE SET t.name = tag.t",
            {{batchSize:5000, iterateList:true, parallel:true}})
        """.format(f=file_name, s=source_label)
        run_query(query, graph, print_only=print_only)
        
## (:Author)-[:HAS_TAG]->(:Tag)
def add_author_tag_edges(authors_files, source_label='', print_only=False):
    logging.info("Adding (:Author{s})-[:HAS_TAG]->(:Tag)".format(s=source_label))
    for file_name in authors_files: 
        query = """
        CALL apoc.periodic.iterate(
        "CALL apoc.load.json('{file}') 
         YIELD value AS q 
         RETURN q",
        "WITH q
         WHERE q.tags IS NOT NULL 
         MATCH (a:Author{source} {{id:q.id}})
         UNWIND q.tags as tag
         MATCH (t:Tag {{cleanName:apoc.text.clean(tag.t)}})
         MERGE (a)-[:HAS_TAG]->(t)",
        {{batchSize:10000, iterateList:true, parallel:false}});
        """.format(file=file_name, source=source_label)
        run_query(query, graph, print_only=print_only)
        
## (:Quanta {idv1})-[:HAS_TAG]->(:Tag)
def add_mag_quanta_tag_edges(papers_v1_files, source_label='', print_only=False):
    logging.info("Adding (:Quanta{s})-[:HAS_TAG]->(:Tag)".format(s=source_label))
    for file_name in authors_files: 
        query = """
        CALL apoc.periodic.iterate(
        "CALL apoc.load.json('{file}') 
         YIELD value AS q 
         RETURN q",
        "WITH q
         WHERE q.keywords IS NOT NULL 
         MATCH (q:Quanta{source} {{id:q.idv1}})
         UNWIND q.keywords as tag
         MERGE (t:Tag {{cleanName:apoc.text.clean(tag.t)}})
         MERGE (q)-[:HAS_TAG]->(t)",
        {{batchSize:5000, iterateList:true, parallel:false}});
        """.format(file=file_name, source=source_label)
        run_query(query, graph, print_only=print_only)
    

# Other updates

In [41]:
def cull_isolated_authors(source_label='', print_only=False):
    logging.info("Removing (a:Author{s}) WHERE not((a)-[:AUTHORED]->())".format(s=source_label))
    query = """
    call apoc.periodic.commit("
    MATCH (a:Author{s}) WHERE NOT ((a)-[:AUTHORED]->())
    WITH a LIMIT {{limit}}
    DETACH DELETE a
    RETURN count(*)
    ", {{limit:50000}});
    """.format(s=source_label)
    run_query(query, graph, print_only=print_only)

def cull_isolated_quanta(source_label='', print_only=False):
    logging.info("Removing (a:Quanta{s}) WHERE not((a)-[:CITES]-())".format(s=source_label))
    query = """
    call apoc.periodic.commit("
    MATCH (a:Quanta{s}) WHERE NOT ((a)-[:CITES]-())
    WITH a LIMIT {{limit}}
    DETACH DELETE a
    RETURN count(*)
    ", {{limit:50000}});
    """.format(s=source_label)
    run_query(query, graph, print_only=print_only)
    
def merge_mag_aminer_venues(print_only=False):
    logging.info("Merging (:Venue:Mag) and (:Venue:Aminer)")
    query = """
    CALL apoc.load.json('oagtwo/venue_linking_pairs.txt') 
    YIELD value as q
    MATCH (m:Venue:Mag {id:q.mid})
    MATCH (a:Venue:Aminer {id:q.aid})
    SET m.idAminer = q.aid
    SET m.idMag = q.mid
    SET m:Aminer:Mag
    WITH [m, a] as ns
    CALL apoc.refactor.mergeNodes(ns, 
         {properties:'discard', mergeRels:true}) 
    YIELD node RETURN 'none'
    """ 
    run_query(query, graph, print_only=print_only)
    
def merge_mag_aminer_authors(print_only=False):
    logging.info("Merging (:Author:Mag) and (:Author:Aminer)")
    query = """
    CALL apoc.periodic.iterate(
    "CALL apoc.load.json('oagtwo/author_linking_pairs.txt') 
     YIELD value as q RETURN q",
    "MATCH (m:Author:Mag {id:q.mid})
     MATCH (a:Author:Aminer {id:q.aid})
     SET m.idAminer = q.aid
     SET m.idMag = q.mid
     SET m:Aminer:Mag
     WITH [m, a] as ns
     CALL apoc.refactor.mergeNodes(ns, 
          {properties:'discard', mergeRels:true}) 
     YIELD node RETURN 'none';",
     {batchSize:10000, iterateList:true, parallel:false});
    """
    run_query(query, graph, print_only=print_only)
    
def merge_mag_aminer_papers(print_only=False):
    logging.info("Merging (:Quanta:Mag) and (:Quanta:Aminer)")
    query = """
    CALL apoc.periodic.iterate(
    "CALL apoc.load.json('oagtwo/paper_linking_pairs.txt') 
     YIELD value as q RETURN q",
    "MATCH (m:Quanta:Mag {id:q.mid})
     MATCH (a:Quanta:Aminer {id:q.aid})
     SET m.idAminer = q.aid
     SET m.idMag = q.mid
     SET m:Aminer:Mag
     WITH [m, a] as ns
     CALL apoc.refactor.mergeNodes(ns, 
          {properties:'discard', mergeRels:true}) 
     YIELD node RETURN 'none';",
     {batchSize:10000, iterateList:true, parallel:false});
    """
    run_query(query, graph, print_only=print_only)

# Run Import

In [46]:
add_quanta_quanta_edges(mag_v1_papers_files, source_label=":Mag", print_only=True)


INFO:root:Adding (:Quanta:Mag)-[:CITES]->(:Quanta:Mag)



        CALL apoc.periodic.commit("
            CALL apoc.load.json('oagone/mag_papers_0.txt') 
            YIELD value AS q
            WITH q LIMIT {limit}",
            MATCH (p:Quanta:Mag {idv1: q.id})
            UNWIND q.references as ref
            WITH p, ref
            MATCH (b:Quanta:Mag {idv1: ref})
            MERGE (p)-[:CITES]->(b)
            ",{limit:10000});
        
Query completed in 0.00 seconds.

        CALL apoc.periodic.commit("
            CALL apoc.load.json('oagone/mag_papers_1.txt') 
            YIELD value AS q
            WITH q LIMIT {limit}",
            MATCH (p:Quanta:Mag {idv1: q.id})
            UNWIND q.references as ref
            WITH p, ref
            MATCH (b:Quanta:Mag {idv1: ref})
            MERGE (p)-[:CITES]->(b)
            ",{limit:10000});
        
Query completed in 0.00 seconds.

        CALL apoc.periodic.commit("
            CALL apoc.load.json('oagone/mag_papers_2.txt') 
            YIELD value AS q
            WITH q LIMIT {li

## Setup database

In [ ]:
add_constraints(print_only=False)
add_indices(print_only=False)
add_years(1700, 2020)

## Import MAG

In [8]:
mag_label = ':Mag'
aminer_label = ':Aminer'

In [15]:
add_venues(mag_venues_file, source_label=mag_label)
add_quanta(mag_papers_files, source_label=mag_label, print_only=True)
add_v1_data(mag_v1_papers_files, source_label=mag_label, print_only=True)
add_authors(mag_authors_files, source_label=mag_label, print_only=True)
add_author_quanta_edges(mag_authors_files, source_label=mag_label, print_only=False)
cull_isolated_authors(source_label=mag_label)
add_organizations(mag_authors_files, source_label=mag_label, print_only=True)
merge_quanta_on_cleanname(source_label=mag_label, print_only=True)
add_quanta_year_edges(mag_papers_files, source_label=mag_label, print_only=False)
add_quanta_venue_edges(mag_papers_files, source_label=mag_label, print_only=False)
add_author_organization_edges(mag_authors_files, source_label=mag_label)
add_quanta_quanta_edges(mag_v1_papers_files, source_label=mag_label)

INFO:root:Merging (q:Quanta:Mag) on q.cleanName



    CALL apoc.periodic.iterate(
        "MATCH (q:Quanta:Mag) 
         WHERE EXISTS(q.cleanName)
         WITH 
             q.cleanName as cleanName, 
             q.year as year, 
             q.citations as citations,
             q.publisher as publisher, 
             COLLECT(q) as nodeList, 
             count(*) as count
         WHERE count>=2
         RETURN nodeList",
        "CALL apoc.refactor.mergeNodes(nodeList, 
         {properties:'discard', mergeRels:true})
         YIELD node
         RETURN 'none'",
        {batchSize:10000, iterateList:true, parallel:false});
    
Query completed in 0.00 seconds.


In [ ]:
add_tags(mag_authors_files, source_label=mag_label)
add_author_tag_edges(mag_authors_files, source_label=mag_label)

## Import AMiner

In [ ]:
add_venues(aminer_venues_file, source_label=aminer_label)
add_quanta(aminer_papers_files, source_label=aminer_label)
add_authors(aminer_authors_files, source_label=aminer_label)
add_author_quanta_edges(aminer_authors_files, source_label=aminer_label)
cull_isolated_authors(source_label=aminer_label)
add_organizations(aminer_authors_files, source_label=aminer_label)
add_quanta_year_edges(aminer_papers_files, source_label=aminer_label)
add_quanta_venue_edges(aminer_papers_files, source_label=aminer_label)
add_author_organization_edges(aminer_authors_files, source_label=aminer_label)
merge_quanta_on_cleanname(source_label=aminer_label, print_only=False)
add_v1_data(aminer_v1_papers_files, source_label=aminer_label)
add_quanta_quanta_edges(aminer_v1_papers_files, source_label=aminer_label)

In [ ]:
add_tags(aminer_authors_files, source_label=aminer_label)
add_author_tag_edges(aminer_authors_files, source_label=aminer_label)

## Further additions and modifications

In [ ]:
add_author_author_edges()

In [ ]:
merge_mag_aminer_venues(venue_linking_pairs)
merge_mag_aminer_papers(paper_linking_pairs)
merge_mag_aminer_authors(author_linking_pairs)

In [ ]:
# MErge duplicate venues, organizations, authors?